# Geo877 Spatial Algorithms - Group Alivand
### Extracting Scenic Routes from VGI Data Sources 
Please enter all your code here, or in the python file with the same name (for classes and other "background" code).